# Приготовления

Этот colab notebook является минимальной демонстрацией для Faceswap GAN.
Поскольку colab допускает максимальное время работы 12 часов, мы будем
обучать только облегченную модель в этом notebook.

**Цель этой notebook - не обучить модель, которая дает высококачественные результаты, а дать краткий обзор того, как работает faceswap-GAN, а также для замены частей лица.**

Порядок работы faceswap-GAN описан ниже:

  1. Загрузите два видео для обучения;
  2. Примените извлечение лица (предварительную обработку) к двум загруженным видео;
  3. Тренируйте легкую модель для Faceswap GAN. (~10-12 ч.)
  4. Примените преобразование видео к загруженным видео.

# Шаг 1: Установите тип среды выполнения на Python 3/GPU
Установите ноутбук colab в экземпляр GPU с помощью:
**runtime -> change runtime type -> Python3 and GPU**

В следующих ячейках будет отображена системная информация текущего экземпляра.
Запустите ячейки и проверьте, использует ли он python > = 3.6 и имеет ли устройство GPU.

Для выполнения кода в colab рекомендуется подключить Google Drive (или Google Storage) для перманентного хранения получаемых данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Создаем директорию для данных на Google Drive или bucket в Google Storage (не реализовано)

Создаем директорию для перманентного сохранения полученных данных в Google Drive или bucket в Google Storage (не реализовано). Colab возвращает ВМ в состояние по умолчанию, если сеанс не активен. Код ниже должен выполняться **единожды**.

In [ ]:
!mkdir -p /content/drive/MyDrive/faceswap_train

In [ ]:
import platform

print(platform.python_version())

Устанавливаем зависимости:

```
tensorflow-gpu==1.15.5
h5py==2.10.0
opencv-python
keras==2.1.5
Keras-Applications
Keras-Preprocessing
imageio==2.4.1
matplotlib
requests
moviepy
Pillow
PyYAML
IPython
```

In [ ]:
# restart the runtime after installation completes
!pip install tensorflow==1.15.5 h5py==2.10.0 keras==2.1.5 imageio==2.4.1 moviepy==0.2.3.5 opencv-python keras_applications matplotlib requests

In [ ]:
%tensorflow_version 1.x

# Шаг 2: Клонируйте репозиторий

In [ ]:
!git clone https://github.com/alvinahmadov/faceswap-parts.git

In [ ]:
!git stash
!git pull

In [ ]:
%cd "faceswap-parts"

In [ ]:
import os
import tensorflow as tf

from tensorflow.python.client import device_lib

print(f"Tensorflow version {tf.version.VERSION}")

device_lib.list_local_devices()

# Шаг 3: Загрузите видеоролики обучения

Пользователь должен загрузить два видео: **source video** и **target video**.
Модель **преобразует исходное лицо в целевое по умолчанию.**

  - Видео, для лучшего результата должен **содержать только одного человека**.
  - Ограничений по длине видео нет, но чем оно длиннее, тем больше времени потребуется
на предварительную обработку/преобразование видео, что может привести к увеличению времени
выполнения до 12 часов. (**Рекомендуемая продолжительность видео: 30 секунд ~ 2 минуты.**)

In [ ]:
try:
    # noinspection PyUnresolvedReferences,PyPackageRequirements
    from google.colab import files
except ImportError:
    print("This notebook can be run only in google colab")
    pass

Можно использовать тестовые видео (выбирать только один способ)

In [ ]:
fn_source_video="samples/source.mp4"
fn_target_video="samples/target.mp4"

Или можно загрузить свои файлы для обработки

In [ ]:
# Upload source video
source_video = files.upload()

for fn_source_video, _ in source_video.items():
    print(fn_source_video)
    pass

In [ ]:
# Upload target video
target_video = files.upload()

for fn_target_video, _ in target_video.items():
    print(fn_target_video)
    pass

# Шаг 5: Все готово.

**Нажмите Ctrl + F10 (или runtime -> run after)**, чтобы запустить процесс. Для завершения
тренировки потребуется 10 ~ 12 часов. Результирующее видео можно загрузить, запустив
последнюю ячейку:
  ```shell
  files.download("OUTPUT_VIDEO.mp4")
  ```
Обратите внимание, что **эту страницу не следует закрывать или обновлять во время работы**.

In [ ]:
%%capture
import imageio

# noinspection PyUnresolvedReferences
imageio.plugins.ffmpeg.download()

In [ ]:
import keras.backend as K
from detector import MTCNNFaceDetector

from preprocess import preprocess_video

In [ ]:
fd = MTCNNFaceDetector(sess=K.get_session(), model_path="./mtcnn_weights/")

Устанавливаем константы для сохранения

In [ ]:
TRAIN_DIR="/content/drive/MyDrive/faceswap_train"

# Path to saved model weights
MODELS_DIR=f"{TRAIN_DIR}/models"
VGGFACE_WEIGHT_FILE=f"{TRAIN_DIR}/models/rcmalli_vggface_tf_notop_resnet50.h5"

# Path to training images
SAVE_PATH_SOURCE=f"{TRAIN_DIR}/face_src"
SAVE_PATH_TARGET=f"{TRAIN_DIR}/face_dst"

Создаем директории для сохранения извлеченных данных, а также для моделей

In [ ]:
from utils import makedirs

In [ ]:
# Create image extraction dirs
for spath in [SAVE_PATH_SOURCE, SAVE_PATH_TARGET]:
  makedirs([f"{spath}/rgb", f"{spath}/binary_mask"])
  pass

In [ ]:
save_interval = 5 # perform face detection every {save_interval} frames

preprocess_video(fn_source_video, fd, save_interval, f"{SAVE_PATH_SOURCE}/")
preprocess_video(fn_target_video, fd, save_interval, f"{SAVE_PATH_TARGET}/")

Получаем количество извлеченных данных

In [ ]:
import glob
face_src_glob_len=str(len(glob.glob(f"{SAVE_PATH_SOURCE}/rgb/*.*")))
face_dst_glob_len=str(len(glob.glob(f"{SAVE_PATH_TARGET}/rgb/*.*")))

print(f"{face_src_glob_len} лиц извлечено из исходного видео: {fn_source_video}.")
print(f"{face_dst_glob_len} лиц извлечено из целевого видео: {fn_target_video}.")

## Следующие ячейки взяты из [faceswap_train_test.ipynb](https://github.com/alvinahmadov/faceswap-parts/blob/main/faceswap_train_test.ipynb)

## Импортируйте пакеты

In [ ]:
import keras.backend as K
import tensorflow as tf

In [ ]:
import os
import glob
import time
from IPython.display import clear_output

%matplotlib inline

## Установите максимальное количество итераций обучения
Для 25000 итераций по умолчанию требуется ~10 часов обучения.

**`Итерации >= 27k могут превышать предельное время выполнения;`**

**`Итерации < 18k могут привести к плохо обученной модели.`**

In [ ]:
TOTAL_ITERS = 21300 # total number of iterations
DISPLAY_ITERS = 300 # each n iters display results

## Конфигурация

In [ ]:
K.set_learning_phase(1)
# Number of CPU cores
num_cpus = os.cpu_count()

# Input/Output resolution
RESOLUTION = 64  # 64x64, 128x128, 256x256
assert (RESOLUTION % 64) == 0, "RESOLUTION should be 64, 128, or 256."

batch_size = 4

# Where to save checkpoints
checkpoint_file=f"{MODELS_DIR}/checkpoint"

# Use motion blur (data augmentation)
# set True if training data contains images extracted from videos
use_da_motion_blur = False

# Use eye-aware training
# require images generated from prep_binary_masks.ipynb
use_bm_eyes = True

# Probability of random color matching (data augmentation)
prob_random_color_match = 0.5

da_config = {
    "prob_random_color_match": prob_random_color_match,
    "use_da_motion_blur": use_da_motion_blur,
    "use_bm_eyes": use_bm_eyes
}

# Path to training images
img_dir_src = f"{SAVE_PATH_SOURCE}/rgb" # source face
img_dir_dst = f"{SAVE_PATH_TARGET}/rgb" # target face
img_dir_src_bm_eyes = f"{SAVE_PATH_SOURCE}/binary_mask"
img_dir_dst_bm_eyes = f"{SAVE_PATH_TARGET}/binary_mask"


# Architecture configuration
arch_config = {
    "IMAGE_SHAPE": (RESOLUTION, RESOLUTION, 3),
    "use_self_attn": True,
    "norm": "hybrid",
    "model_capacity": "lite"
}

# Loss function weights configuration
loss_weights = {
    "w_D": 0.1,
    "w_recon": 1.,
    "w_edge": 0.1,
    "w_eyes": 30.,
    "w_pl": (0.01, 0.1, 0.3, 0.1)
}

# Init. loss config.
loss_config = {
    "gan_training": "mixup_LSGAN",
    "use_PL": False,
    "PL_before_activ": True,
    "use_mask_hinge_loss": False,
    "m_mask": 0.,
    "lr_factor": 1.,
    "use_cyclic_loss": False
}

## Постройте модель

In [ ]:
from nn.faceswap_model import FaceswapModel
from loader import DataLoader

In [ ]:
model = FaceswapModel(**arch_config)

Скачиваем модель из `rcmalli_vggface_tf_notop_resnet50` и сохраняем в директории моделей `.../faceswap_train/models`

In [ ]:
%%capture
!wget https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_tf_notop_resnet50.h5 -P /content/drive/MyDrive/faceswap_train/models

In [ ]:
from nn.vggface import RESNET50

vggface = RESNET50(include_top=False, weights=None, input_shape=(224, 224, 3))
vggface.load_weights(VGGFACE_WEIGHT_FILE, by_name=True)

model.build_pl_model(vggface_model=vggface, before_activ=loss_config["PL_before_activ"])
model.build_train_functions(loss_weights=loss_weights, **loss_config)

## Начните тренировку

In [ ]:
# Create ./models directory
makedirs([MODELS_DIR])

In [ ]:
# Get file names
train_src = glob.glob(f"{img_dir_src}/*.*")
train_dst = glob.glob(f"{img_dir_dst}/*.*")

train_src_n_dst = train_src + train_dst

assert len(train_src), f"Изображение не найдено в {img_dir_src}"
assert len(train_dst), f"Изображение не найдено в {img_dir_dst}"
print(f"Количество изображений в папке A: {str(len(train_src))}")
print(f"Количество изображений в папке B: {str(len(train_dst))}")

Создаем класс для записи/чтения контрольных точек для непрерывного обучения.

In [ ]:
from utils import CheckPoint, save_image, save_loss_data, showG, showG_mask

In [ ]:
def show_loss_config(loss_conf):
    for config, value in loss_conf.items():
        print(f"{config} = {value}")
        pass
    pass

Показываем события для этапов обучения

In [ ]:
def show_conditions(gen_iterations, cond=None):
    info_msg = "Выполняется условие "
    cond_msg0 = f"0: gen_iterations % display_iters == {(gen_iterations % DISPLAY_ITERS) == 0}"
    cond_msg1 = f"1: gen_iterations == {(TOTAL_ITERS // 5 - DISPLAY_ITERS // 2)}"
    cond_msg2 = f"2: gen_iterations == {(TOTAL_ITERS // 5 + TOTAL_ITERS // 10 - DISPLAY_ITERS // 2)}"
    cond_msg3 = f"3: gen_iterations == {(2 * TOTAL_ITERS // 5 - DISPLAY_ITERS // 2)}"
    cond_msg4 = f"4: gen_iterations == {(TOTAL_ITERS // 2 - DISPLAY_ITERS // 2)}"
    cond_msg5 = f"5: gen_iterations == {(2 * TOTAL_ITERS // 3 - DISPLAY_ITERS // 2)}"
    cond_msg6 = f"6: gen_iterations == {(8 * TOTAL_ITERS // 10 - DISPLAY_ITERS // 2)}"
    cond_msg7 = f"7: gen_iterations == {(9 * TOTAL_ITERS // 10 - DISPLAY_ITERS // 2)}"

    if cond is None:
        print(f"Условия:\n\t{cond_msg1}\n\t{cond_msg2}\n\t{cond_msg3}\n\t{cond_msg4}"
              f"\n\t{cond_msg5}\n\t{cond_msg6}\n\t{cond_msg7}\n\t{cond_msg0}\n\n")
        pass
    elif cond == 0:
        print(info_msg + cond_msg0)
        pass
    elif cond == 1:
        print(info_msg + cond_msg1)
        pass
    elif cond == 2:
        print(info_msg + cond_msg2)
        pass
    elif cond == 3:
        print(info_msg + cond_msg3)
        pass
    elif cond == 4:
        print(info_msg + cond_msg4)
        pass
    elif cond == 5:
        print(info_msg + cond_msg5)
        pass
    elif cond == 6:
        print(info_msg + cond_msg6)
        pass
    elif cond == 7:
        print(info_msg + cond_msg7)
        pass
    pass

Сбрасываем сессию

In [ ]:
def reset_session(spath):
    """
    Parameters
    ----------
    spath : str
     Save path
    """
    global model, vggface
    global train_batch_src, train_batch_dst
    model.save_weights(path=spath)
    del model
    del vggface
    del train_batch_src
    del train_batch_dst
    K.clear_session()
    model = FaceswapModel(**arch_config)
    model.load_weights(path=spath)
    vggface = RESNET50(include_top=False, weights=None, input_shape=(224, 224, 3))
    vggface.load_weights(VGGFACE_WEIGHT_FILE)
    model.build_pl_model(vggface_model=vggface, before_activ=loss_config["PL_before_activ"])
    train_batch_src = DataLoader(filenames=train_src, all_filenames=train_src_n_dst,
                                 batch_size=batch_size, dir_bm_eyes=img_dir_src_bm_eyes,
                                 resolution=RESOLUTION, num_cpus=num_cpus, session=K.get_session(),
                                 **da_config)
    train_batch_dst = DataLoader(filenames=train_dst, all_filenames=train_src_n_dst,
                                 batch_size=batch_size, dir_bm_eyes=img_dir_dst_bm_eyes,
                                 resolution=RESOLUTION, num_cpus=num_cpus, session=K.get_session(),
                                 **da_config)
    pass

In [ ]:
TRAIN_RESULTS_DIR=f"{TRAIN_DIR}/train"


Определяем функцию обучения.

Примечание: При использовании обучения на tensorflow-cpu мин. время выполнения блока кода составляло ~ 15 сек.
При ипользовании tensorflow-gpu то же время составило ~ 1 сек.

Для оптимальных результатов (как по времени, так и по качеству) рекомендуется значение
`TOTAL_ITERS == 18 000`

In [ ]:
def train(checkpoint):
    clear_output()
    # Start training
    t0 = time.time()

    model_dir=f"{MODELS_DIR}"

    # This try/except is meant to resume training if we disconnected from Colab

    try:
      gen_iterations
      print(f"Возобновить обучение c {gen_iterations} итерации.")
      checkpoint.save(gen_iterations, t0)
    except:
      gen_iterations = checkpoint.load().iter
      print(f"Возобновить обучение c {gen_iterations} итерации.")
      pass

    def show_cond(cond=None):
        show_conditions(gen_iterations, cond)

    errGA_sum = errGB_sum = errDA_sum = errDB_sum = 0
    errGAs = {}
    errGBs = {}

    for k in ['ttl', 'adv', 'recon', 'edge', 'pl']:
        errGAs[k] = 0
        errGBs[k] = 0
        pass

    global TOTAL_ITERS
    global train_batch_src, train_batch_dst

    train_batch_src = DataLoader(train_src, train_src_n_dst, batch_size,
                                 dir_bm_eyes=img_dir_src_bm_eyes, resolution=RESOLUTION,
                                 num_cpus=num_cpus, session=K.get_session(), **da_config)

    train_batch_dst = DataLoader(train_dst, train_src_n_dst, batch_size,
                                 dir_bm_eyes=img_dir_dst_bm_eyes, resolution=RESOLUTION,
                                 num_cpus=num_cpus, session=K.get_session(), **da_config)

    show_cond()
    save_path = ""
    while gen_iterations <= TOTAL_ITERS:
        # Loss function automation
        if (gen_iterations + 1) % DISPLAY_ITERS == 0:
            print(f"Выполняется итерация {gen_iterations}/{TOTAL_ITERS}")
        exec_time = time.time()

        # condition1
        if gen_iterations == (TOTAL_ITERS // 5 - DISPLAY_ITERS // 2):
            show_cond(1)
            loss_config['use_PL'] = True
            loss_config['use_mask_hinge_loss'] = False
            loss_config['m_mask'] = 0.0
            reset_session(model_dir)
            print("Конструкция новых функций потерь...")
            show_loss_config(loss_config)
            model.build_train_functions(loss_weights=loss_weights, **loss_config)
            t1 = time.time() - exec_time
            print("Условие %i выполнено за %.2f сек." % (1, t1))
            pass
        # condition2
        elif gen_iterations == (TOTAL_ITERS // 5 + TOTAL_ITERS // 10 - DISPLAY_ITERS // 2):
            show_cond(2)
            loss_config['use_PL'] = True
            loss_config['use_mask_hinge_loss'] = True
            loss_config['m_mask'] = 0.5
            reset_session(model_dir)
            print("Конструкция новых функций потерь...")
            show_loss_config(loss_config)
            model.build_train_functions(loss_weights=loss_weights, **loss_config)
            t1 = time.time() - exec_time
            print("Условие %i выполнено за %.2f сек." % (2, t1))
            pass
        # condition3
        elif gen_iterations == (2 * TOTAL_ITERS // 5 - DISPLAY_ITERS // 2):
            show_cond(3)
            loss_config['use_PL'] = True
            loss_config['use_mask_hinge_loss'] = True
            loss_config['m_mask'] = 0.2
            reset_session(model_dir)
            print("Конструкция новых функций потерь...")
            show_loss_config(loss_config)
            model.build_train_functions(loss_weights=loss_weights, **loss_config)
            t1 = time.time() - exec_time
            print("Условие %i выполнено за %.2f сек." % (3, t1))
            pass
        # condition4
        elif gen_iterations == (TOTAL_ITERS // 2 - DISPLAY_ITERS // 2):
            show_cond(4)
            loss_config['use_PL'] = True
            loss_config['use_mask_hinge_loss'] = True
            loss_config['m_mask'] = 0.4
            loss_config['lr_factor'] = 0.3
            reset_session(model_dir)
            print("Конструкция новых функций потерь...")
            show_loss_config(loss_config)
            model.build_train_functions(loss_weights=loss_weights, **loss_config)
            t1 = time.time() - exec_time
            print("Условие %i выполнено за %.2f сек." % (4, t1))
            pass
        # condition5
        elif gen_iterations == (2 * TOTAL_ITERS // 3 - DISPLAY_ITERS // 2):
            show_cond(5)
            model.decoder_src.load_weights(f"{model_dir}/decoder_B.h5")  # swap decoders
            model.decoder_dst.load_weights(f"{model_dir}/decoder_A.h5")  # swap decoders
            loss_config['use_PL'] = True
            loss_config['use_mask_hinge_loss'] = True
            loss_config['m_mask'] = 0.5
            loss_config['lr_factor'] = 1
            reset_session(model_dir)
            print("Конструкция новых функций потерь...")
            show_loss_config(loss_config)
            model.build_train_functions(loss_weights=loss_weights, **loss_config)
            t1 = time.time() - exec_time
            print("Условие %i выполнено за %.2f сек." % (5, t1))
            pass
        # condition6
        elif gen_iterations == (8 * TOTAL_ITERS // 10 - DISPLAY_ITERS // 2):
            show_cond(6)
            loss_config['use_PL'] = True
            loss_config['use_mask_hinge_loss'] = True
            loss_config['m_mask'] = 0.1
            loss_config['lr_factor'] = 0.3
            reset_session(model_dir)
            print("Конструкция новых функций потерь...")
            show_loss_config(loss_config)
            model.build_train_functions(loss_weights=loss_weights, **loss_config)
            t1 = time.time() - exec_time
            print("Условие %i выполнено за %.2f сек." % (6, t1))
            pass
        # condition7
        elif gen_iterations == (9 * TOTAL_ITERS // 10 - DISPLAY_ITERS // 2):
            show_cond(7)
            loss_config['use_PL'] = True
            loss_config['use_mask_hinge_loss'] = False
            loss_config['m_mask'] = 0.0
            loss_config['lr_factor'] = 0.1
            reset_session(model_dir)
            print("Конструкция новых функций потерь...")
            show_loss_config(loss_config)
            model.build_train_functions(loss_weights=loss_weights, **loss_config)
            t1 = time.time() - exec_time
            print("Условие %i выполнено за %.2f сек." % (7, t1))
            pass

        # Train dicriminators for one batch
        data_src = train_batch_src.get_next_batch()
        data_dst = train_batch_dst.get_next_batch()
        errDA, errDB = model.train_one_batch_disc(data_src, data_dst)
        errDA_sum += errDA[0]
        errDB_sum += errDB[0]

        # Train generators for one batch
        data_src = train_batch_src.get_next_batch()
        data_dst = train_batch_dst.get_next_batch()
        errGA, errGB = model.train_one_batch_gen(data_src, data_dst)
        errGA_sum += errGA[0]
        errGB_sum += errGB[0]
        for i, k in enumerate(['ttl', 'adv', 'recon', 'edge', 'pl']):
            errGAs[k] += errGA[i]
            errGBs[k] += errGB[i]
            pass

        gen_iterations += 1

        # Visualization
        if gen_iterations % DISPLAY_ITERS == 0:
            show_cond(0)

            # Display loss information
            show_loss_config(loss_config)

            train_info="[iter %d] time: %f\nLoss_DA: %f Loss_DB: %f\nLoss_GA: %f Loss_GB: %f" % (
                gen_iterations, time.time() - t0, errDA_sum / DISPLAY_ITERS, errDB_sum / DISPLAY_ITERS,
                errGA_sum / DISPLAY_ITERS, errGB_sum / DISPLAY_ITERS
            )
            train_info=f"\n{train_info}\n###################################"

            adv_loss_info = f"[Adversarial loss]\nGA: {errGAs['adv'] / DISPLAY_ITERS:.4f} GB: {errGBs['adv'] / DISPLAY_ITERS:.4f}"
            rec_loss_info = f"[Reconstruction loss]\nGA: {errGAs['recon'] / DISPLAY_ITERS:.4f} GB: {errGBs['recon'] / DISPLAY_ITERS:.4f}"
            edg_loss_info = f"[Edge loss]\nGA: {errGAs['edge'] / DISPLAY_ITERS:.4f} GB: {errGBs['edge'] / DISPLAY_ITERS:.4f}"
            prc_loss_info = ""

            try:
                prc_loss_info = f"[Perceptual loss]\nGA: {errGAs['pl'][0] / DISPLAY_ITERS:.4f} GB: {errGBs['pl'][0] / DISPLAY_ITERS:.4f}"
            except:
                prc_loss_info = f"[Perceptual loss]\nGA: {errGAs['pl'] / DISPLAY_ITERS:.4f} GB: {errGBs['pl'] / DISPLAY_ITERS:.4f}"
                pass

            # print(train_info)
            # print("Детали потерь генератора:")
            # print(adv_loss_info)
            # print(rec_loss_info)
            # print(edg_loss_info)

            # if loss_config['use_PL']:
            #     print(prc_loss_info)
            #     pass

            loss_info = f"{train_info}\n{adv_loss_info}\n{rec_loss_info}\n{edg_loss_info}\n{prc_loss_info}"

            save_path = f"{TRAIN_RESULTS_DIR}/{gen_iterations}"
            makedirs([save_path])
            save_loss_data(save_path, loss_info)

            w_src, t_src, _ = train_batch_src.get_next_batch()
            w_dst, t_dst, _ = train_batch_dst.get_next_batch()

            # Save images
            print("Преобразованные результаты:")
            save_image(t_src, t_dst, model.path_src, model.path_dst, batch_size,
                       im_save_path=save_path, filename="preproc.png")

            showG(t_src, t_dst, model.path_src, model.path_dst, batch_size)
            print("Маски:")
            save_image(t_src, t_dst, model.path_mask_src, model.path_mask_dst,
                       batch_size, im_save_path=save_path, filename="bm.png", is_mask=True)
            showG_mask(t_src, t_dst, model.path_mask_src, model.path_mask_dst, batch_size)
            print("Результаты реконструкции:")
            save_image(w_src, w_dst, model.path_bgr_src, model.path_bgr_dst, batch_size,
                       im_save_path=save_path, filename="recon.png")
            showG(w_src, w_dst, model.path_bgr_src, model.path_bgr_dst, batch_size)
            errGA_sum = errGB_sum = errDA_sum = errDB_sum = 0
            for k in ['ttl', 'adv', 'recon', 'edge', 'pl']:
                errGAs[k] = 0
                errGBs[k] = 0
                pass

            # Save models
            model.save_weights(path=model_dir)

            t1 = time.time() - exec_time
            print("Сохранение модели выполнено за %.2f сек." % (t1))
            pass

        t1 = time.time() - exec_time
        checkpoint.save(gen_iterations, t1)
        print("Выполнение блока кода завершено за %.2f сек. (осталось ~%.2f сек.)\n" % (t1, (TOTAL_ITERS - gen_iterations)*t1))
        pass
    pass

Начинаем обучение.

Благодаря системе записи/чтения состояний обучения, а также сохранения результатов, операцию
обучения можно выполнять прерывно, даже если произошло исключение или остановка процесса.
При каждом завершении обучения для текущего `TOTAL_ITERS` итераций, можно улучшить результат
повышая значение `TOTAL_ITERS`, но при условии, если **`TOTAL_ITERS` % `DISPLAY_ITERS` == 0**

In [ ]:
checkpoint = CheckPoint(checkpoint_file)

try:
    train(checkpoint)
except:
    checkpoint.save(checkpoint.load().iter + 1)
    pass

print("Training finished at checkpoint " + str(checkpoint))

# Конвертация видео
[faceswap_video_conversion.ipynb](https://github.com/alvinahmadov/faceswap-parts/blob/main/colab/faceswap_video_conversion.ipynb)

In [ ]:
from converter import Converter
from converter.config import ConverterConfig, TransformDirection, ImageOutputType

In [ ]:
global model, vggface
global train_batch_src, train_batch_dst
try:
  del model
  del vggface
  del train_batch_src
  del train_batch_dst
except NameError:
  pass
tf.reset_default_graph()
K.clear_session()

In [ ]:
from converter.config import ColorCorrectionType

config = ConverterConfig(
    use_smoothed_bbox=True, use_kalman_filter=True,
    use_auto_downscaling=False, bbox_moving_avg_coef=0.65,
    min_face_area=35 * 35, kf_noise_coef=1e-3, color_correction=ColorCorrectionType.HISTMATCH,
    detection_threshold=0.8, roi_coverage=0.9, enhance=0.0,
    output_type=ImageOutputType.TRIPLE, direction=TransformDirection.AtoB
)

In [ ]:
#fd = MTCNNFaceDetector(sess=K.get_session(), model_path="./mtcnn_weights/")
vc = Converter("../mtcnn_weights/", model_path=MODELS_DIR, session=K.get_session(), config=config)

In [ ]:
if config.direction == TransformDirection.AtoB:
    input_fn = fn_source_video
    output_fn = "OUTPUT_VIDEO_AtoB.mp4"
    pass
elif config.direction == TransformDirection.BtoA:
    input_fn = fn_target_video
    output_fn = "OUTPUT_VIDEO_BtoA.mp4"
    pass

duration = None  # None or a non-negative float tuple: (start_sec, end_sec). Duration of input video to be converted

In [ ]:

session = K.get_session()
with session.as_default():
    with session.graph.as_default():
        vc.convert(input_fn, output_fn, duration=duration, audio=True)
        pass
    pass

# Скачать результат (видеофайл)

In [ ]:
try:
    # noinspection PyUnresolvedReferences,PyPackageRequirements
    from google.colab import files
except ImportError:
    print("This notebook can be run only in google colab")
    pass

In [ ]:
if config.direction == TransformDirection.AtoB:
    files.download("OUTPUT_VIDEO_AtoB.mp4")
    pass
elif config.direction == TransformDirection.BtoA:
    files.download("OUTPUT_VIDEO_BtoA.mp4")
    pass